In [420]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math

In [421]:
df401k = pd.read_csv('401k.csv')
# print(len(df401k))

dfTable = pd.DataFrame(columns=["Income Range",
                                "Count",
                                "First_Stage",
                                "Net_Financial_Asset_OLS", 
                                "Net_Financial_Asset_2SLS", 
                                "Net_non401k_Asset_OLS",
                                "Net_non401k_Asset_2SLS",
                                "Total_Wealth_OLS",
                                "Total_Wealth_2SLS"]
                                )
dfTable["Income Range"] = ["All","less_than_10k", "10k_20k", "20k_30k","30k_40k","40k_50k","50k-75k","more_than_75k"]

Available covariates are:
1. p401 : participation in 401(k)
2. e401 : eligibility for 401(k)
3. a401 : 401(k) assets
4. tw : total wealth (in US $)
5. tfa : financial assets (in US $)
6. net_tfa : net financial assets (in US $)
7. nifa : non-401k financial assets (in US $)
8. net_nifa : net non-401k financial assets
9. net_n401 : net non-401(k) assets (in US $)
10. ira : individual retirement account (IRA)
11. inc : income (in US $)
12. age : age
13. fsize : family size
14. marr : married
15. pira : participation in IRA
16. db : defined benefit pension
17. hown : home owner
18. educ : education (in years)
19. male : male
20. twoearn : two earners
21. nohs, hs, smcol, col : dummies for education: no high-school, high-school, some
college, college
22. hmort : home mortage (in US $)
23. hequity : home equity (in US $)
24. hval : home value (in US $)

Cov: age, income, family size, education, marital status, two-earner status, defined benefit (DB) pension status, IRA participation status, and homeownership
status

In [422]:
# Process data
# Income catergory (<10K, 10–20K, 20–30K, 30–40K, 40–50K, 50–75K, ≥75K)
df401k["all"] = ((df401k["inc"]>=float('-Inf')) & (df401k["inc"]<float('Inf'))).astype(int)
df401k["i10k"] = (df401k["inc"]<10E3).astype(int)
df401k["i10_20k"] = ((df401k["inc"]>=10E3) & (df401k["inc"]<20E3)).astype(int)
df401k["i20_30k"] = ((df401k["inc"]>=20E3) & (df401k["inc"]<30E3)).astype(int)
df401k["i30_40k"] = ((df401k["inc"]>=30E3) & (df401k["inc"]<40E3)).astype(int)
df401k["i40_50k"] = ((df401k["inc"]>=40E3) & (df401k["inc"]<50E3)).astype(int)
df401k["i50_75k"] = ((df401k["inc"]>=50E3) & (df401k["inc"]<=75E3)).astype(int)
df401k["i75k"] = (df401k["inc"]>=75E3).astype(int)

# Age catergory (<30, 30–35, 36–44, 45–54, ≥55)
df401k["a30"] = (df401k["age"]<30).astype(int)
df401k["a30_35"] = ((df401k["age"]>=30) & (df401k["age"]<=35)).astype(int)
df401k["a36_44"] = ((df401k["age"]>=36) & (df401k["age"]<=44)).astype(int)
df401k["a45_54"] = ((df401k["age"]>=45) & (df401k["age"]<=54)).astype(int)
df401k["a55"] = (df401k["age"]>=55).astype(int)

# Education catergory (<12,12,13-15,>=16)
df401k["el12"] = (df401k["educ"]<12).astype(int)
df401k["e12"] = (df401k["educ"]==12).astype(int)
df401k["e12_16"] = ((df401k["educ"]>12) & (df401k["educ"]<16)).astype(int)
df401k["e16"] = (df401k["educ"]>=16).astype(int)

df401k["e401"]=df401k["e401"].astype(int)
df401k["marr"]=df401k["marr"].astype(int)
df401k["twoearn"]=df401k["twoearn"].astype(int)
df401k["pira"]=df401k["pira"].astype(int)
df401k["db"]=df401k["db"].astype(int)
df401k["hown"]=df401k["hown"].astype(int)

print(df401k.shape[0])  # 9915
print(df401k["e401"].mean())    # 0.37 (eligibility rate)
print(df401k["p401"].mean())
print(df401k["net_tfa"].mean())
print(df401k["net_n401"].mean())
print(df401k["tw"].mean()) 

# print(df401k["p401"].astype(int))
df401k_original = df401k

# The lowest interval dummy variabls for inc,age,educ are natural baselines and is dropped from regression 
# (to avoid dummy variable trap where multicolinearity arise)
income_list = ["all","i10k","i10_20k","i20_30k","i30_40k","i40_50k","i50_75k","i75k"]
y_list = ["net_tfa","net_n401","tw"]
X_list = [  "a30_35","a36_44","a45_54","a55",
            "twoearn",
            "fsize",
            "marr",
            "pira",
            "db",
            "hown",
            "e12","e12_16","e16"]

inc_dummy = ["i10_20k","i20_30k","i30_40k","i40_50k","i50_75k","i75k"]

9915
0.3713565305093293
0.2616238023197176
18051.534846192637
13877.020675743823
63816.84659606656


In [423]:
column_name_OLS = ["Net_Financial_Asset_OLS","Net_non401k_Asset_OLS","Total_Wealth_OLS"]
column_name_2SLS = ["Net_Financial_Asset_2SLS","Net_non401k_Asset_2SLS","Total_Wealth_2SLS"]

for ind,i in enumerate(income_list):

    # Take the rows for the given interval
    idx = df401k_original.index[df401k_original.loc[:,i]==1]
    df401k = df401k_original.loc[idx]
    dfTable.loc[ind,"Count"]=len(list(idx))

    # Dummy variables if all income, linear inc if interval income
    loc_inc_var = ["inc"] if i != "all" else inc_dummy
  
    # OLS covariate matrix
    X0 = df401k[["p401"]+loc_inc_var+X_list]
    X0 = sm.add_constant(X0)

    # 1) 2SLS First stage: D on Z and X
    X1 = sm.add_constant(df401k[["e401"]+loc_inc_var+X_list])
    fs = sm.OLS(df401k["p401"],X1).fit()
    D_hat = fs.fittedvalues
    dfTable.loc[ind,"First_Stage"]=f'{fs.params["e401"]:.3f} ({fs.bse["e401"]:.3f})'

    # Create 2SLS Second stage covariate matrix
    X2 = sm.add_constant(pd.concat([D_hat.rename('D_hat'),df401k[loc_inc_var+X_list]], axis=1))

    for y,col_ols,col_2sls in zip(y_list,column_name_OLS,column_name_2SLS):
        # OLS alpha
        ols_model_ntfa = sm.OLS(df401k[[y]], X0).fit(cov_type='HC1')  # HC1 = robust SE
        D = ols_model_ntfa.params["p401"]
        bse = ols_model_ntfa.bse["p401"]
        dfTable.loc[ind,col_ols]=f'{D:.0f} ({bse:.0f})' if math.floor(D)!=0 else f'{D:.3f}({bse:.0f})'

        # 2SLS alpha
        # 2) Second stage: y on D_hat and X
        s2 = sm.OLS(df401k[[y]], X2).fit(cov_type='HC1')
        D2 = s2.params["D_hat"]
        bse2 = s2.bse["D_hat"]
        dfTable.loc[ind,col_2sls]=f'{D2:.0f} ({bse2:.0f})' if math.floor(D2)!=0 else f'{D2:.3f}({bse2:.0f})'

dfTable


,Income Range,Count,First_Stage,Net_Financial_Asset_OLS,Net_Financial_Asset_2SLS,Net_non401k_Asset_OLS,Net_non401k_Asset_2SLS,Total_Wealth_OLS,Total_Wealth_2SLS
0,All,9915,0.697 (0.006),14520 (1551),13087 (1927),778 (1477),-355 (1855),10694 (2388),9259 (3038)
1,less_than_10k,638,0.711 (0.020),9843 (4921),9149 (4967),4093 (3447),3443 (3537),20464 (11311),17224 (11659)
2,10k_20k,1948,0.650 (0.013),5591 (1463),5352 (1642),-759 (1227),-917 (1428),4729 (2265),6138 (3214)
3,20k_30k,2074,0.627 (0.013),7083 (1315),4143 (2284),448 (1124),-2518 (2145),5462 (3119),0.183(4502)
4,30k_40k,1712,0.672 (0.015),12136 (2513),10273 (2901),1077 (2297),-909 (2677),10683 (3891),4881 (5113)
5,40k_50k,1204,0.723 (0.018),12858 (2470),9980 (3766),500 (2330),-2479 (3642),13470 (4905),13205 (6684)
6,50k-75k,1572,0.744 (0.017),20800 (3010),21920 (3460),1803 (2876),2985 (3309),12281 (5132),12202 (6724)
7,more_than_75k,767,0.831 (0.022),23104 (10417),24013 (12899),-6735 (10228),-5252 (12715),5514 (13645),10470 (17169)
